In [473]:
import pandas as pd

### Download the data and load it to Pandas. 

You can find them [here](https://drive.google.com/file/d/1NY6cmF9Shjw-dD7BD6bNmfcIVz-kQcFR/view?usp=sharing).

In [474]:
sales1 = pd.read_csv('_data/sales1.csv')
sales2 = pd.read_csv('_data/sales2.csv')
sales1

,Book title,Number sold,Sales price,Royalty paid
0,The Bricklayer’s Bible,8,2.99,0.55
1,Swimrand,2,1.99,0.35
2,Pining For The Fisheries of Yore,28,2.99,0.55
3,The Duck Goes Here,34,2.99,0.55
4,The Tower Commission Report,4,11.50,4.25


## Challenge: 

#### 1. Combine these two sales CSV together into a single dataframe.
#### 2. Then compute how much money consumers spent on each book in each currency.

**Notes:**
- First table is in USD
- Second table actually consists of 3 smaller tables. First 9 rows are for USD, next 8 rows are for GBP, and final 7 are for EUR.

In [475]:
sales2[sales2['Title'].str.contains('royalties').fillna(False)]['Title']

8         US royalties (USD)
16        UK royalties (GBP)
23    France royalties (EUR)
Name: Title, dtype: object

In [476]:
sales1['Total price'] = sales1['Sales price'] * sales1['Number sold']

In [477]:
royalty_dividers = sales2[sales2['Title'].str.contains('royalties').fillna(False)].index

In [478]:
USD_sales = sales2.loc[:royalty_dividers[0], :]
GBP_sales = sales2.loc[royalty_dividers[0]:royalty_dividers[1], :]
EUR_sales = sales2.loc[royalty_dividers[1]:royalty_dividers[2], :]
USD_sales['Total price USD'] = USD_sales['Units sold'] * USD_sales['List price']
GBP_sales['Total price GBP'] = GBP_sales['Units sold'] * GBP_sales['List price']
EUR_sales['Total price EUR'] = EUR_sales['Units sold'] * EUR_sales['List price']

/var/folders/xm/yv4_9lnx4vxb435678j65d140000gn/T/ipykernel_57796/1078971136.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  USD_sales['Total price USD'] = USD_sales['Units sold'] * USD_sales['List price']
/var/folders/xm/yv4_9lnx4vxb435678j65d140000gn/T/ipykernel_57796/1078971136.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GBP_sales['Total price GBP'] = GBP_sales['Units sold'] * GBP_sales['List price']
/var/folders/xm/yv4_9lnx4vxb435678j65d140000gn/T/ipykernel_57796/1078971136.py:6: SettingWithC

In [479]:
USD_sales_books = USD_sales[USD_sales['Title'].isin(sales1['Book title'])].drop(columns=['Units sold', 'List price', 'Royalty'])
GBP_sales_books = GBP_sales[GBP_sales['Title'].isin(sales1['Book title'])].drop(columns=['Units sold', 'List price', 'Royalty'])
EUR_sales_books = EUR_sales[EUR_sales['Title'].isin(sales1['Book title'])].drop(columns=['Units sold', 'List price', 'Royalty'])

In [480]:
USD_sales_books

,Title,Total price USD
4,Swimrand,2.99
6,The Duck Goes Here,101.66
7,The Tower Commission Report,38.00


In [481]:
GBP_sales_books

,Title,Total price GBP
15,The Tower Commission Report,26.0


In [482]:
EUR_sales_books

,Title,Total price EUR
21,Swimrand,15.92
22,The Duck Goes Here,23.88


In [483]:
sales1

,Book title,Number sold,Sales price,Royalty paid,Total price
0,The Bricklayer’s Bible,8,2.99,0.55,23.92
1,Swimrand,2,1.99,0.35,3.98
2,Pining For The Fisheries of Yore,28,2.99,0.55,83.72
3,The Duck Goes Here,34,2.99,0.55,101.66
4,The Tower Commission Report,4,11.50,4.25,46.00


In [484]:
sales = pd.merge(sales1[['Book title', 'Total price']], USD_sales_books, left_on=['Book title'], right_on=['Title'], how='outer').fillna(0)
sales['Total price USD'] = sales['Total price'] + sales['Total price USD']
sales.drop(columns=['Title', 'Total price'], inplace=True)

In [485]:
sales = pd.merge(sales, GBP_sales_books,left_on=['Book title'], right_on=['Title'], how='outer').fillna(0)
sales.drop(columns='Title', inplace=True)
sales['Total price GBP'] = sales['Total price GBP'].apply(lambda x: "{:.2f}".format(x))

In [486]:
sales = pd.merge(sales, EUR_sales_books,left_on=['Book title'], right_on=['Title'], how='outer').fillna(0)
sales.drop(columns='Title', inplace=True)

In [487]:
sales

,Book title,Total price USD,Total price GBP,Total price EUR
0,The Bricklayer’s Bible,23.92,0.00,0.00
1,Swimrand,6.97,0.00,15.92
2,Pining For The Fisheries of Yore,83.72,0.00,0.00
3,The Duck Goes Here,203.32,0.00,23.88
4,The Tower Commission Report,84.00,26.00,0.00


In [488]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Book title       5 non-null      object 
 1   Total price USD  5 non-null      float64
 2   Total price GBP  5 non-null      object 
 3   Total price EUR  5 non-null      float64
dtypes: float64(2), object(2)
memory usage: 292.0+ bytes
